<a href="https://colab.research.google.com/github/Saba-Gul/FastAPI-Sentiment-Analysis-and-Text-Summarization-API/blob/main/Creating_Endpoints_Using_Fast_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Running FastAPI on Google Colab
We will be going through a quick demo on how to run a basic fastAPI server on google colab.

### What is FastAPI?
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.
The key features are:

*  Fast: Very high performance, on par with NodeJS and Go (thanks to Starlette and
*  Pydantic). One of the fastest Python frameworks available.
*  Fast to code: Increase the speed to develop features by about 200% to 300%.
*  Fewer bugs: Reduce about 40% of human (developer) induced errors. *
*  Intuitive: Great editor support. Completion everywhere. Less time debugging.
*  Easy: Designed to be easy to use and learn. Less time reading docs.
*  Short: Minimize code duplication. Multiple features from each parameter declaration. Fewer bugs.
*  Robust: Get production-ready code. With automatic interactive documentation.
*  Standards-based: Based on (and fully compatible with) the open standards for APIs

Let's now jump to FastAPI which is a very promising library of Python due to the fact that it is based on ASGI and handles request asynchronously increasing throughput and performance as compared to Flask that used WSGI which doesn't support async calls. FastAPI has a good documentation available out there as well. In order to run FastAPI on Google Colab, we need to install it first along with the following packages:

*  nest-asyncio : This presents a practical problem: When in an environment where the event loop is already running it’s impossible to run tasks and wait for the result. Trying to do so will give the error “RuntimeError: This event loop is already running”. The issue pops up in various environments, such as web servers, GUI applications and in Jupyter notebooks. This module patches asyncio to allow nested use of asyncio.run and loop.run_until_complete.
*  pyngrok : This is a Python wrapper for ngrok that manages its own binary and puts it on your path, making ngrok readily available from anywhere on the command line and via a convenient Python API. ngrok is made even more powerful with native Python integration through pyngrok.
*  uvicorn : This includes a Gunicorn worker class allowing you to run ASGI applications, with all of Uvicorn's performance benefits, while also giving you Gunicorn's fully-featured process management.

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.7 MB/s eta 0:00:00


We import *FastAPI* class from fastapi module and create its object that we call, *app*. We also create a route with @app.route('/index') decorator which is linked with the async *home()* method. *async* acts as a modifier to make the function execute asynchronously. The local FastAPI server runs at port number 8000 and ngrok creates a tunnel so that outside networks can also access it as described earlier in the flask server implementation. Since FastAPI supports async calls we need a ASGI server i.e. uvicorn which runs the app at port number 8000 locally.

In [ ]:
!ngrok config add-authtoken 'add the token here without the quotes'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install transformers

# **Creating an End point for sentiment classification**

In [ ]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model="cardiffnlp/twitter-roberta-base-sentiment-latest")
results = classifier(["I love you", "I hate you"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'label': 'positive', 'score': 0.8594695329666138}, {'label': 'negative', 'score': 0.7866932153701782}]


In [ ]:
print(results)

[{'label': 'positive', 'score': 0.8594695329666138}, {'label': 'negative', 'score': 0.7866932153701782}]


In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import joblib
from transformers import pipeline
#import requests
app = FastAPI()

sentiment = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

def sentiment_model(text_input):
  return str(sentiment(text_input))

def get_sentiment_score(user_input):
  classifier = pipeline('sentiment-analysis', model="cardiffnlp/twitter-roberta-base-sentiment-latest")
  results = classifier([user_input])

  return str(results)

#our_trained_model =joblib.load("/content/our_model.sav")

@app.get('/')
async def home():

  return "Welcome to Our FastAPI Endpoints"

@app.get('/test')
async def test():

  return "This our second Endpoint, we call it Test"

@app.get('/sentiment/{user_input}')
async def sentiment(user_input):

  return get_sentiment_score(user_input)

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [917]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://0944-34-75-115-238.ngrok-free.app
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET / HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /openapi.json HTTP/1.1" 200 OK


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /sentiment/I%20am%20happy HTTP/1.1" 200 OK


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /sentiment/I%20am%20not%20happy HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [917]


# **Use a Hugging Face Transformer to summarize text and find its sentiment**

**Text Summarization pipeline**

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
input_text = """Hugging Face: Revolutionizing Natural Language Processing Introduction In the rapidly evolving field of Natural Language Processing (NLP), Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face, a company that has made remarkable contributions to NLP and AI as a whole. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry.  The Birth of Hugging Face Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf. The name Hugging Face was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug. Initially, they began as a chatbot company but later shifted their focus to NLP, driven by their belief in the transformative potential of this technology. Transformative Innovations Hugging Face is best known for its open-source contributions, particularly the Transformers library. This library has become the de facto standard for NLP and enables researchers, developers, and organizations to easily access and utilize state-of-the-art pre-trained language models, such as BERT, GPT-3, and more. These models have countless applications, from chatbots and virtual assistants to language translation and sentiment analysis. """
pipe = pipeline("summarization", model="Falconsai/text_summarization")


Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf . The name was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug .


In [ ]:
pipe(input_text)

[{'summary_text': "Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf . The name was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug ."}]

In [ ]:
print(pipe(input_text)[0]['summary_text'])

Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf . The name was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug .


**Text Classification of summarized text pipeline**

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
var = "Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf . The name was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug ."
emotion = pipeline("text-classification", model="michellejieli/emotion_text_classifier")

In [ ]:
emotion(var)

[{'label': 'neutral', 'score': 0.897263765335083}]

In [ ]:
emotion(var)[0]['label']

'neutral'

**Combining the pipeline**

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

input_text = """Hugging Face: Revolutionizing Natural Language Processing Introduction In the rapidly evolving field of Natural Language Processing (NLP), Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face, a company that has made remarkable contributions to NLP and AI as a whole. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry.  The Birth of Hugging Face Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf. The name Hugging Face was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug. Initially, they began as a chatbot company but later shifted their focus to NLP, driven by their belief in the transformative potential of this technology. Transformative Innovations Hugging Face is best known for its open-source contributions, particularly the Transformers library. This library has become the de facto standard for NLP and enables researchers, developers, and organizations to easily access and utilize state-of-the-art pre-trained language models, such as BERT, GPT-3, and more. These models have countless applications, from chatbots and virtual assistants to language translation and sentiment analysis. """
summarization = pipeline("summarization", model="Falconsai/text_summarization")
emotion = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Summarize text->text/emotion classification of summarized text
emotion(summarization(input_text)[0]['summary_text'])[0]['label']

'neutral'

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from transformers import pipeline

summarization = pipeline("summarization", model="Falconsai/text_summarization")
emotion = pipeline("text-classification", model="michellejieli/emotion_text_classifier")

sentiment = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

app = FastAPI()

def sentiment_model(text_input):
  return str(sentiment(text_input))


@app.get('/sentiment/{input_text}')
async def sentiment_analysis(input_text):
  return sentiment_model(input_text)


@app.get('/ai/{input_text}') # summarize and emotion classification
async def AI_wala_fun(input_text):
  #input_text = """Hugging Face: Revolutionizing Natural Language Processing Introduction In the rapidly evolving field of Natural Language Processing (NLP), Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face, a company that has made remarkable contributions to NLP and AI as a whole. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry.  The Birth of Hugging Face Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf. The name Hugging Face was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug. Initially, they began as a chatbot company but later shifted their focus to NLP, driven by their belief in the transformative potential of this technology. Transformative Innovations Hugging Face is best known for its open-source contributions, particularly the Transformers library. This library has become the de facto standard for NLP and enables researchers, developers, and organizations to easily access and utilize state-of-the-art pre-trained language models, such as BERT, GPT-3, and more. These models have countless applications, from chatbots and virtual assistants to language translation and sentiment analysis. """

  return emotion(summarization(input_text)[0]['summary_text'])[0]['label']

def alg_se_fun(number): # print a table
  result = []
  for i in range(1,10):

    result.append(int(i*number))

  return result

@app.get('/')
async def home():

  return "Welcome to Our FastAPI Endpoints"

@app.get('/atomcamp')
async def atom():

  return "Welcome to Our 2nd Endpoints, it seems fun, and easy"

@app.get('/{number}')
async def table(number):

  return str(alg_se_fun(int(number)))


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

Public URL: https://4487-34-75-115-238.ngrok-free.app


INFO:     Started server process [917]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2001:1970:53c2:2c00::1feb:0 - "GET / HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /favicon.ico HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 398, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-pack

INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /sentiment/I%20hate%20it HTTP/1.1" 200 OK


Your max_length is set to 200, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /ai/i%20hate%20it HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [917]


# **Custom LLM Endpoint Using OpenAI**

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=key)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [ ]:
print(response.choices[0].message.content)

The 2020 World Series was played at Globe Life Field in Arlington, Texas. This was the first time the World Series was held at a neutral site due to the COVID-19 pandemic.


In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from transformers import pipeline

summarization = pipeline("summarization", model="Falconsai/text_summarization")
emotion = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
sentiment = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

from openai import OpenAI
client = OpenAI(api_key=key)
def chat(prompt):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt }
    ]
  )
  return response.choices[0].message.content


app = FastAPI()

def sentiment_model(text_input):
  return str(sentiment(text_input))


@app.get('/llm/{input_text}')
async def llm_chat(input_text):
  return chat(input_text)

@app.get('/sentiment/{input_text}')
async def sentiment_analysis(input_text):
  return sentiment_model(input_text)


@app.get('/ai/{input_text}')
async def AI_wala_fun(input_text):
  #input_text = """Hugging Face: Revolutionizing Natural Language Processing Introduction In the rapidly evolving field of Natural Language Processing (NLP), Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face, a company that has made remarkable contributions to NLP and AI as a whole. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry.  The Birth of Hugging Face Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf. The name Hugging Face was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug. Initially, they began as a chatbot company but later shifted their focus to NLP, driven by their belief in the transformative potential of this technology. Transformative Innovations Hugging Face is best known for its open-source contributions, particularly the Transformers library. This library has become the de facto standard for NLP and enables researchers, developers, and organizations to easily access and utilize state-of-the-art pre-trained language models, such as BERT, GPT-3, and more. These models have countless applications, from chatbots and virtual assistants to language translation and sentiment analysis. """

  return emotion(summarization(input_text)[0]['summary_text'])[0]['label']

def alg_se_fun(number):
  result = []
  for i in range(1,10):

    result.append(int(i*number))

  return result

@app.get('/')
async def home():

  return "Welcome to Our FastAPI Endpoints"

@app.get('/atomcamp')
async def atom():

  return "Welcome to Our 2nd Endpoints, it seems fun, and easy"

@app.get('/{number}')
async def table(number):

  return str(alg_se_fun(int(number)))


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Public URL: https://718c-34-81-252-222.ngrok-free.app


INFO:     Started server process [423]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2001:1970:53c2:2c00::1feb:0 - "GET / HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /favicon.ico HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 398, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-pack

INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2001:1970:53c2:2c00::1feb:0 - "GET /llm/when%20do%20kids%20usually%20start%20talking HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [423]
